## File Preprocessing

### Text Preprocessing
- remove digits
- remove punctuations
- tokenize sentences
- remove stop words
- stemming tokens
- lemmatize words

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer

### Preprocessing using nltk pagages

In [4]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer

def nltkPreprocessing(text):
    
    #remove digits
    dig_translator = str.maketrans('', '', '0123456789')
    text = text.translate(dig_translator)
    
    #remove punctuation
    str_translator = str.maketrans('', '', string.punctuation)
    text = text.translate(str_translator).lower()
    text = text.strip()
    
    #tokenize sentences
    word_tokens = word_tokenize(text)
    stop_words = stopwords.words('german')
    
    #remove stop words
    filtered_tokens = [w.lower() for w in word_tokens if not w.lower() in stop_words]
    
    #using PorterStemmer to stem the tokens (effect not good)
#     ps = SnowballStemmer('german')
#     stem_tokens = [ps.stem(w) for w in filtered_tokens]
    
    return filtered_tokens
#     return stem_tokens



### Preprocessing using spacy packages

In [3]:
import spacy
nlp = spacy.load('de')

def spacyPreprocessing(text): 
    #remove digits
    dig_translator = str.maketrans('', '', '0123456789-/€®–')
    text = text.translate(dig_translator)
    #tokenize texts
    word_tokens = nlp(text.lower())
    word_tokens = [w for w in word_tokens if not w.is_stop] #remove stop words
    word_tokens = [w for w in word_tokens if not w.is_punct]#remove punctuation
    word_tokens = [w for w in word_tokens if not w.is_space]#remove space
    word_tokens = [w for w in word_tokens if not w.is_currency]#remove currency
    word_tokens = [w.lemma_ for w in word_tokens if not w.like_num]#remove num like, lemmatize words
    return word_tokens

/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [152]:
######### Toy Example #############

# import json
# path = './sample/BMW-AnnualReport-2015.json'
# word_list = []
# with open(path, 'r') as f:
#     for line in f:
#         obj = json.loads(line)
#         textType = obj['type']
#         if textType == 'paragraph':
#             words = spacyPreprocessing(obj['content'])
#             word_list +=words